<a href="https://colab.research.google.com/github/abpelegrina/SketchEngineAPI-intro/blob/master/BasicQuerySKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Manual de python para SKE


Instalar los paquetes necesarios. Para ellos ejecuta el siguiente comando:

In [0]:
!pip install pandas numpy xlrd

Importamos los paquetes que vamos a usar:

In [0]:
import requests
import pandas as pd
import time
import datetime

Asignamos los datos de acceso:

In [0]:
USERNAME = ''
API_KEY = ''

Definimos el corpus:

In [0]:
corpus = ''

Definimos la URL para las consultas: 

In [0]:
base_url = 'https://api.sketchengine.eu/bonito/run.cgi'
q = 'view'
url = base_url + '/' + q + '?corpname=' + corpus

Declaramos la consulta a realizar:

In [0]:
query = '[tag!=\"N.*|JJ.*\"] [lemma=\"plastic\"] [lemma=\"water\"] [tag=\"JJ.*|N.*|RB.*|VVG.*|VVN.*\" & lemma!= \"bottle\"]* [tag=\"N.*\" & lemma!= \"bottle\"] within <s/>'

Construimos los datos que vamos a enviar a SketchEngine

In [0]:
data = {
 'format': 'json',
 'q': ['q'+query],
 'async': 0
}

Mandamos la consulta a SKE (tarda un poco...):

In [0]:
response = requests.get(url, params=data, auth=(USERNAME, API_KEY))

Comprobamos que la consulta se haya realizado con éxito y mostramos los resultados.


Si se ha producido algún error al efectuar la consulta los mostramos por pantalla.

In [0]:
if response:
    # guardamos el resultado que nos ha
    d = response.json()
    
    # Mostramos el resultado por pantalla. Esto se puede borrar para que se muestre
    # solo la frecuencia
    print('Resultado de la consulta:')
    print(d)

    # Devolvemos sólo el parámetro que contiene la frecuencia de la consulta
    print('\nFrecuencia de la consulta {' + query + '}: ')
    print(d['fullsize'])

# Si se produce algún error lo mostramos
else:
    print(response.text)
    print(response.headers)

Resultado de la consulta:
{'fullsize': 0, 'Lines': [], 'request': {'q': 'q[tag!="N.*|JJ.*"] [lemma="plastic"] [lemma="water"] [tag="JJ.*|N.*|RB.*|VVG.*|VVN.*" & lemma!= "bottle"]* [tag="N.*" & lemma!= "bottle"] within <s/>', 'corpname': 'user/LRG00/eolica_en_melania', 'async': '0', 'format': 'json'}, 'finished': 1, 'async': 0, 'concsize': 0, 'api_version': '4.24.9', 'Desc': [{'nicearg': 'N.*|JJ.*, plastic, water, JJ.*|N.*|RB.*|VVG.*|VVN.*, bottle, N.*', 'tourl': 'q=q%5Btag%21%3D%22N.%2A%7CJJ.%2A%22%5D+%5Blemma%3D%22plastic%22%5D+%5Blemma%3D%22water%22%5D+%5Btag%3D%22JJ.%2A%7CN.%2A%7CRB.%2A%7CVVG.%2A%7CVVN.%2A%22+%26+lemma%21%3D+%22bottle%22%5D%2A+%5Btag%3D%22N.%2A%22+%26+lemma%21%3D+%22bottle%22%5D+within+%3Cs%2F%3E', 'rel': 0.0, 'arg': '[tag!="N.*|JJ.*"] [lemma="plastic"] [lemma="water"] [tag="JJ.*|N.*|RB.*|VVG.*|VVN.*" & lemma!= "bottle"]* [tag="N.*" & lemma!= "bottle"] within <s/>', 'op': 'Query', 'size': 0}]}

Frecuencia de la consulta {[tag!="N.*|JJ.*"] [lemma="plastic"] [lemma="w

Cambiamos la consulta para usar el parámetro filter-hits:

In [0]:
data = {
 'format': 'json',
 'q': ['q'+query,'D'], # Si no quieres el parámetro
 'async': 0
}


Repetimos la consulta y mostramos el número de resultados:

In [0]:
response = requests.get(url, params=data, auth=(USERNAME, API_KEY))
if response:
    # guardamos el resultado que nos ha
    d = response.json()

    # Devolvemos sólo el parámetro que contiene la frecuencia de la consulta
    print('\nFrecuencia de la consulta {' + query + '}: ')
    print(d['fullsize'])

# Si se produce algún error lo mostramos
else:
    print(response.text)
    print(response.headers)


Frecuencia de la consulta {[tag!="N.*|JJ.*"] [lemma="plastic"] [lemma="water"] [tag="JJ.*|N.*|RB.*|VVG.*|VVN.*" & lemma!= "bottle"]* [tag="N.*" & lemma!= "bottle"] within <s/>}: 
0




Tiempo en segundos que hay que esperar entre consultas para no agotar la cuota diaria
¡¡CAMBIAR SI VE VAN A HACER MUCHAS CONSULTAS!! Seguir esta guía:
   - if you want to make fewer than 50 requests, time_to_sleep = 0
   - if you want to make up to 900 requests, time_to_sleep = 4
   - if you want to make more than 2000 requests, time_to_sleep = 44


In [0]:
time_to_sleep = 0

Ahora vamos a abrir un fichero con unas pocas consultas


In [0]:
df = pd.read_excel('original.xlsx')
df_res = df.copy()
df.head()

,p1,p2,p3,p1p2,p2p3,p1p3
0,offshore,wind,farm,"[tag!=""JJ.*|N.*""][lemma=""offshore""][lemma=""win...","[tag!=""JJ.*|N.*""][lemma=""wind""][lemma=""farm""][...","[tag!=""JJ.*|N.*""][lemma=""offshore""][lemma=""far..."
1,tip,speed,ratio,"[tag!=""JJ.*|N.*""][lemma=""tip""][lemma=""speed""][...","[tag!=""JJ.*|N.*""][lemma=""speed""][lemma=""ratio""...","[tag!=""JJ.*|N.*""][lemma=""tip""][lemma=""ratio""][..."
2,wind,power,plant,"[tag!=""JJ.*|N.*""][lemma=""wind""][lemma=""power""]...","[tag!=""JJ.*|N.*""][lemma=""power""][lemma=""plant""...","[tag!=""JJ.*|N.*""][lemma=""wind""][lemma=""plant""]..."


Definimos las filas por las que vamos a comenzar y a terminar

In [0]:
# Empezamos las consultas desde la primera fila
start = 0

# Terminamos con la última fila
end = df.shape[0]

Recorremos la tabla por filas, ejecutando la consulta de cada celda. Cuando terminemos mostramos el resultado por pantalla.

In [0]:
def getFrequency(query):

    # Imprimimos por pantalla la consulta
    print(query)
    
    # Información a enviar a SKE
    data = {
     'format': 'json',
     # Eliminar ", 'D'" si no se quiere usar el parámtro filter_hits
     'q': ['q'+query,'D'],
     'async': 0
    }
    
    # Imprimimos el tiempo en el que se realiza la consulta
    print(datetime.datetime.now().time())
    
    # Realizamos la consulta
    response = requests.get(url, params=data, auth=(USERNAME, API_KEY))
    
    # Si la consulta a SKE se ha realizado con éxito
    if response:
            # Obtenemos el tiempo de consulta
            elapsed = response.elapsed.total_seconds()
            
            # guardamos el resultado que nos ha
            d = response.json()
            
            # Si la consulta ha durado menos que el tiempo de espera entre consultas "dormimos" la diferencia
            if elapsed < time_to_sleep:
                time.sleep(time_to_sleep-elapsed)
                
            # Devolvemos sólo el parámetro que contiene la frecuencia de la consulta
            return d['fullsize']
            
    # Si se produce algún error devolvemos la consulta para que se mantenga la consulta
    # en la celda
    else:
        print(response.text) # mostramos el error
        print(response.headers) # mostramos el error
        return query


for index in range(start, end):
    # Imprimimos por pantalla el número de la fila
    print('Row Number : ', index)
    
    # Para cada celda de la fila (excluyendo las tres primeras que tienen sólo los términos)
    # calculamos la frecuencia de la consulta y la guardamos
    df_res.iloc[index, 3:] = df.iloc[index, 3:].apply(lambda x: getFrequency(x) )

df_res.head()

Row Number :  0
[tag!="JJ.*|N.*"][lemma="offshore"][lemma="wind"][tag!="N.*|JJ.*"] within <s/>
18:55:38.403593
[tag!="JJ.*|N.*"][lemma="wind"][lemma="farm"][tag!="N.*|JJ.*"] within <s/>
18:55:39.159863
[tag!="JJ.*|N.*"][lemma="offshore"][lemma="farm"][tag!="N.*|JJ.*"] within <s/>
18:55:39.923579
Row Number :  1
[tag!="JJ.*|N.*"][lemma="tip"][lemma="speed"][tag!="N.*|JJ.*"] within <s/>
18:55:40.650856
[tag!="JJ.*|N.*"][lemma="speed"][lemma="ratio"][tag!="N.*|JJ.*"] within <s/>
18:55:41.344519
[tag!="JJ.*|N.*"][lemma="tip"][lemma="ratio"][tag!="N.*|JJ.*"] within <s/>
18:55:42.116724
Row Number :  2
[tag!="JJ.*|N.*"][lemma="wind"][lemma="power"][tag!="N.*|JJ.*"] within <s/>
18:55:42.838420
[tag!="JJ.*|N.*"][lemma="power"][lemma="plant"][tag!="N.*|JJ.*"] within <s/>
18:55:43.561859
[tag!="JJ.*|N.*"][lemma="wind"][lemma="plant"][tag!="N.*|JJ.*"] within <s/>
18:55:44.341639


,p1,p2,p3,p1p2,p2p3,p1p3
0,offshore,wind,farm,426,2648,29
1,tip,speed,ratio,39,19,1
2,wind,power,plant,2601,240,156
